# TP 5-6 : Partie 2 : Problème avec précédences et ressources disjonctives

Initialization (à faire une seule fois)

In [ ]:
#import Pkg; Pkg.add("Clp")

## Relaxation des contraintes de ressources

In [1]:
D=[6, 7, 0, 3, 5, 1]

using JuMP
using Clp

model = Model(Clp.Optimizer) # set optimizer
set_optimizer_attribute(model, "LogLevel", 0) #don't display anything during solve
set_optimizer_attribute(model, "Algorithm", 4) #LP solver chosen is simplex

# define t variables
@variable(model, t[i in 1:6] >= 0)
@variable(model, tfin)

# define objective function
@objective(model, Min, tfin)

# define constraints: t_j - t_i  >= D[i], \forall i predecesseur de j
@constraint(model, t[2] - t[1] >= D[1] )
@constraint(model, t[3] - t[2] >= D[2] )
@constraint(model, t[5] - t[4] >= D[4] )
@constraint(model, t[6] - t[5] >= D[5] )

#define constraints: tfin - t_i >= Duree[i], \forall i predecesseur de j
@constraint(model, tfin - t[3] >= D[3] )
@constraint(model, tfin - t[6] >= D[6] )

println(model)

print("start solve ... ")
optimize!(model)
print("... end solve")


println("\n\nSolution PL:\n \t t=", value.(t), "\t tfin=", value(tfin))

Min tfin
Subject to
 -t[1] + t[2] >= 6.0
 -t[2] + t[3] >= 7.0
 -t[4] + t[5] >= 3.0
 -t[5] + t[6] >= 5.0
 -t[3] + tfin >= 0.0
 -t[6] + tfin >= 1.0
 t[1] >= 0.0
 t[2] >= 0.0
 t[3] >= 0.0
 t[4] >= 0.0
 t[5] >= 0.0
 t[6] >= 0.0

start solve ... ... end solve

Solution PL:
 	 t=[0.0, 6.0, 13.0, 0.0, 3.0, 8.0]	 tfin=13.0


In [2]:
import Pkg
using LinearAlgebra

# Fonction calculant le maximum d'un tableau
function maximumTableau(tableau)
    taille = size(tableau)[1]
    maximum = tableau[1]
    indice = 1
    for i in 1:taille
          if (tableau[i] > maximum)
            maximum = tableau[i]
            indice = i
        end
    end
    return maximum, indice
end

# Fonction retournant la liste des prédcesseurs d'un noeud dans un graphe
function Predecesseur(graphe,j)
    nbSommets = size(graphe)[1]
    listePredecesseurs = []
    for i in 1:nbSommets
        # Si le noeud a une valeur autre que 0, Inf ou - Inf, on l'ajoute 
        # à la liste des prédécesseurs du noeud dont on recherche les 
        # prédécesseurs
        if graphe[i,j]!= Inf && graphe[i,j]!= -Inf
            listePredecesseurs = append!([i],listePredecesseurs)
        end
    end
    return listePredecesseurs
end

function BellmanFordLong(graphe, source)
    
    # Nombre de sommets du graphe
    nbSommets = size(graphe)[1]

    # Distance parcourue pour chaque
    distance =  fill(-Inf,(nbSommets + 1,nbSommets))
    distance[1, source]=0

    k=1
    bool=true
    
    # L'algorithme de Bellman-Ford s'arrête dès que 2 lignes du tableau de
    # suite sont égales ou que k vaut nbSommets + 1
    while (bool && k<nbSommets+1)
        # On incrémente k
        k += 1
        # La ligne k est initialisée à partie de la ligne k-1
        distance[k, :] = distance[k-1, :]
        for i in 2:nbSommets
            # On récupère les prédécesseurs de i
            prede_i= Predecesseur(graphe,i)
            l=[]
            for j in prede_i
                # on ajoute la somme du poids du predecesseur et de sa distance
                l=append!([distance[k-1,j]+graphe[j,i]],l)
            end
            # On récupère le maximum de la liste précédente
            distance[k,i], indice = maximumTableau(l)
        end
        # Si cette ligne du tableau est la même que la précédente on s'arrête
        if (distance[k-1, :]==distance[k, :])
                bool=false
        end
    end
    # On retourne la dernière ligne
    return distance[k,:]
end

graphe = 
[#debut a1    a2     a3     b1    b2      b3     fin
  0     0     -Inf   -Inf   0     -Inf    -Inf   -Inf ;
 -Inf   0     6      -Inf   -Inf   -Inf   -Inf   -Inf ;
 -Inf   -Inf  0      7      -Inf   -Inf    -Inf   -Inf ;
 -Inf   -Inf  -Inf   0      -Inf   -Inf    -Inf   0 ;
 -Inf   -Inf  -Inf   -Inf   0    3      -Inf   -Inf ;
 -Inf   -Inf  -Inf   -Inf   -Inf 0      5      -Inf ;
 -Inf   -Inf  -Inf   -Inf   -Inf -Inf   0      1 ;
 -Inf   -Inf  -Inf   -Inf   -Inf -Inf   -Inf   0 ;
]

distance = BellmanFordLong(graphe, 1)
println("Date de début de [t1,t2,t3,t4,t5,t6]: ", distance[2:length(distance) - 1])
println("Date de fin : ", distance[length(distance)])

Date de début de [t1,t2,t3,t4,t5,t6]: [0.0, 6.0, 13.0, 0.0, 3.0, 8.0]
Date de fin : 13.0


Récupération des données

In [3]:
MachinesParJob=[1 3 0; 1 2 3; 3 2 1]
Duree=[6 7 0; 3 5 1; 1 1 1]
bigM=sum(Duree)

25

Procédure d'application des tests de sondabilités TA, TO et TR pour le cas de la relaxation linéaire

In [4]:
function TestsSondabilite_relaxlin(model2, varsbin, BestTfin, Bestsol)
    TA, TO, TR = false, false, false
    if(termination_status(model2) == MOI.INFEASIBLE)#Test de faisabilite
        TA=true
        println("TA")
    elseif(objective_value(model2) >= BestTfin) #Test d'optimalite
        TO=true
        println("TO")
    elseif( prod(abs.([round.(v, digits=0) for v in value.(varsbin)]-value.(varsbin)) .<= fill(10^-5, size(varsbin))) 
        ) #Test de resolution
        TR=true
        println("TR")
        if (value(tfin) <= BestTfin)
            Bestsol = value.(t)
            BestTfin=value(tfin)
        end
    else
        println("non sondable")
    end
    TA, TO, TR, Bestsol, BestTfin
end

TestsSondabilite_relaxlin (generic function with 1 method)

Procédure de séparation et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [5]:
function SeparerNoeud_relaxlin(varsshouldbebinary, listvars, listvals)
    # le noeud est non-sondable. Appliquer le critère de séparation pour le séparer en sous-noeuds 
    # et choisir un noeud-fils le plus à gauche   
    
    #find a fractionnal variable
    i, var = 1, 0
    while((i <= length(varsshouldbebinary)) && (var==0))
        #if(varsshouldbebinary[i] ∉ listvars)
        if(abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) ) >= 10^-5)
            var=varsshouldbebinary[i]
        end
        i+=1
    end
    
    #=
    #find most fractionnal variable
    i, var, maxfrac = -1, 0, 0.0
    for i in 1:length(varsshouldbebinary)
        if(abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) ) >= maxfrac) 
            #if a variable is more fractinonal
            var=varsshouldbebinary[i]
            maxfrac=abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) )
            #println(i, " ", var, " ", maxfrac)
        end
    end
    =#
    

    set_lower_bound(var,1.0)
    set_upper_bound(var,1.0)

    push!(listvars,var) #stocker l'identite de la variable choisie pour la séparation
    push!(listvals,1.0) #stocker la branche choisie, identifiee par la valeur de la variable choisie
    listvars, listvals
end


function ExplorerAutreNoeud_relaxlin(listvars, listvals)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    #go back to parent node
    var=pop!(listvars)
    theval=pop!(listvals)
    set_lower_bound(var,0.0)
    set_upper_bound(var,1.0)

    #go to right child if possible, otherwise go back to parent
    while( (theval==0.0) && (length(listvars)>= 1))
        var=pop!(listvars)
        theval=pop!(listvals)
        set_lower_bound(var,0.0) 
        set_upper_bound(var,1.0)
    end
    if theval==1.0
        set_lower_bound(var,0.0)
        set_upper_bound(var,0.0)
        push!(listvars,var)
        push!(listvals,0.0)
    else
        println("\nFINISHED")
        stop=true
    end
    listvars, listvals, stop 
end

ExplorerAutreNoeud_relaxlin (generic function with 1 method)

Creation de la relaxation linéaire (= modèle associé au noeud 0)

In [6]:
# ROOT NODE
using JuMP
using Clp

model2 = Model(Clp.Optimizer) # set optimizer
set_optimizer_attribute(model2, "LogLevel", 0) #don't display anything during solve
set_optimizer_attribute(model2, "Algorithm", 4) #LP solver chosen is simplex

# define t variables
@variable(model2, t[i in 1:3, j in 1:3] >= 0)
@variable(model2, tfin)

# define objective function
@objective(model2, Min, tfin)

# define constraints: t_i(j+1) - t_ij  >= Duree[i,j], \forall i,j
@constraint(model2, t[1,2] - t[1,1] >= Duree[1,1] )
@constraint(model2, t[1,3] - t[1,2] >= Duree[1,2] )
@constraint(model2, t[2,2] - t[2,1] >= Duree[2,1] )
@constraint(model2, t[2,3] - t[2,2] >= Duree[2,2] )

#define constraints: tfin - t_ij >= Duree[i,j], \forall ij
@constraint(model2, tfin - t[1,3] >= Duree[1,3] )
@constraint(model2, tfin - t[2,3] >= Duree[2,3] )
@constraint(model2, tfin - t[3,3] >= Duree[3,3] )


# define x variables as CONTINUOUS (recall that it is not possible to define binary variables in Clp)
@variable(model2, 0 <= x_2_1__1_1 <= 1)
@variable(model2, 0 <= x_2_3__1_2 <= 1)
@variable(model2, 0 <= x_3_1__2_2 <= 1)
varsshouldbebinary=[x_2_1__1_1,x_2_3__1_2,x_3_1__2_2]


# define bigM constraints linking x and t variables
@constraint(model2, t[2,1] - t[1,1] >=  Duree[1,1] - bigM*x_2_1__1_1)
@constraint(model2, t[1,1] - t[2,1] >=  Duree[2,1] - bigM*(1-x_2_1__1_1))
@constraint(model2, t[2,3] - t[1,2] >=  Duree[1,2] - bigM*x_2_3__1_2)
@constraint(model2, t[1,2] - t[2,3] >=  Duree[2,3] - bigM*(1-x_2_3__1_2))
@constraint(model2, t[3,1] - t[2,2] >=  Duree[2,2] - bigM*(1-x_3_1__2_2))
@constraint(model2, t[2,2] - t[3,1] >=  Duree[3,1] - bigM*x_3_1__2_2)


println(model2)

Min tfin
Subject to
 -t[1,1] + t[1,2] >= 6.0
 -t[1,2] + t[1,3] >= 7.0
 -t[2,1] + t[2,2] >= 3.0
 -t[2,2] + t[2,3] >= 5.0
 -t[1,3] + tfin >= 0.0
 -t[2,3] + tfin >= 1.0
 -t[3,3] + tfin >= 1.0
 -t[1,1] + t[2,1] + 25 x_2_1__1_1 >= 6.0
 t[1,1] - t[2,1] - 25 x_2_1__1_1 >= -22.0
 -t[1,2] + t[2,3] + 25 x_2_3__1_2 >= 7.0
 t[1,2] - t[2,3] - 25 x_2_3__1_2 >= -24.0
 t[3,1] - t[2,2] - 25 x_3_1__2_2 >= -20.0
 -t[3,1] + t[2,2] + 25 x_3_1__2_2 >= 1.0
 t[1,1] >= 0.0
 t[2,1] >= 0.0
 t[3,1] >= 0.0
 t[1,2] >= 0.0
 t[2,2] >= 0.0
 t[3,2] >= 0.0
 t[1,3] >= 0.0
 t[2,3] >= 0.0
 t[3,3] >= 0.0
 x_2_1__1_1 >= 0.0
 x_2_3__1_2 >= 0.0
 x_3_1__2_2 >= 0.0
 x_2_1__1_1 <= 1.0
 x_2_3__1_2 <= 1.0
 x_3_1__2_2 <= 1.0



In [7]:
listvars=[]
listvals=[]

BestTfin=bigM
Bestsol=[]

current_node_number=0
stop = false

while(!stop)
    
    println("\nNode number ", current_node_number, ": \n-----\n", model2)

    print("Solve : start ... ")
    status = optimize!(model2)
    println("... end")

    println("\nSolution relax lin",); [print("\t", name(v),"=",value(v)) for v in all_variables(model2)]; 
    println(" "); println("\nSolution precedemment memorisee ", Bestsol, " avec date de fin ", BestTfin, "\n")

    TA, TO, TR, Bestsol, BestTfin = TestsSondabilite_relaxlin(model2, varsshouldbebinary, BestTfin, Bestsol)

    is_node_sondable = TA || TO || TR
    
    if(!is_node_sondable)
        listvars, listvals = SeparerNoeud_relaxlin(varsshouldbebinary, listvars, listvals)
    else
        listvars, listvals, stop = ExplorerAutreNoeud_relaxlin(listvars, listvals)
    end

    current_node_number = current_node_number + 1
end

println("\n******\n\nOptimal value = ", BestTfin, "\n\nOptimal t=", Bestsol)


Node number 0: 
-----
Min tfin
Subject to
 -t[1,1] + t[1,2] >= 6.0
 -t[1,2] + t[1,3] >= 7.0
 -t[2,1] + t[2,2] >= 3.0
 -t[2,2] + t[2,3] >= 5.0
 -t[1,3] + tfin >= 0.0
 -t[2,3] + tfin >= 1.0
 -t[3,3] + tfin >= 1.0
 -t[1,1] + t[2,1] + 25 x_2_1__1_1 >= 6.0
 t[1,1] - t[2,1] - 25 x_2_1__1_1 >= -22.0
 -t[1,2] + t[2,3] + 25 x_2_3__1_2 >= 7.0
 t[1,2] - t[2,3] - 25 x_2_3__1_2 >= -24.0
 t[3,1] - t[2,2] - 25 x_3_1__2_2 >= -20.0
 -t[3,1] + t[2,2] + 25 x_3_1__2_2 >= 1.0
 t[1,1] >= 0.0
 t[2,1] >= 0.0
 t[3,1] >= 0.0
 t[1,2] >= 0.0
 t[2,2] >= 0.0
 t[3,2] >= 0.0
 t[1,3] >= 0.0
 t[2,3] >= 0.0
 t[3,3] >= 0.0
 x_2_1__1_1 >= 0.0
 x_2_3__1_2 >= 0.0
 x_3_1__2_2 >= 0.0
 x_2_1__1_1 <= 1.0
 x_2_3__1_2 <= 1.0
 x_3_1__2_2 <= 1.0

Solve : start ... ... end

Solution relax lin
	t[1,1]=0.0	t[2,1]=0.0	t[3,1]=2.0000000000000004	t[1,2]=6.0	t[2,2]=3.0	t[3,2]=0.0	t[1,3]=13.0	t[2,3]=8.000000000000002	t[3,3]=0.0	tfin=13.0	x_2_1__1_1=0.24	x_2_3__1_2=0.2	x_3_1__2_2=0.0 

Solution precedemment memorisee Any[] avec date de fin 

Boucle principale : résoudre la relaxation linéaire, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [8]:
println("\n******\n\nOptimal value = ", BestTfin, "\n\nOptimal t=", Bestsol)


******

Optimal value = 14.999999999999998

Optimal t=[0.0 6.0 14.999999999999998; 6.0 9.0 13.999999999999998; 8.0 0.0 0.0]


In [9]:
MachinesParJob=[1 3 2; 1 2 3]
Duree=[6 7 4; 3 5 1]
bigM=sum(Duree)

26

In [10]:
# ROOT NODE
using JuMP
using Clp

model2 = Model(Clp.Optimizer) # set optimizer
set_optimizer_attribute(model2, "LogLevel", 0) #don't display anything during solve
set_optimizer_attribute(model2, "Algorithm", 4) #LP solver chosen is simplex

# define t variables
@variable(model2, t[i in 1:2, j in 1:3] >= 0)
@variable(model2, tfin)

# define objective function
@objective(model2, Min, tfin)

# define constraints: t_i(j+1) - t_ij  >= Duree[i,j], \forall i,j
@constraint(model2, t[1,2] - t[1,1] >= Duree[1,1] )
@constraint(model2, t[1,3] - t[1,2] >= Duree[1,2] )
@constraint(model2, t[2,2] - t[2,1] >= Duree[2,1] )
@constraint(model2, t[2,3] - t[2,2] >= Duree[2,2] )

#define constraints: tfin - t_ij >= Duree[i,j], \forall ij
@constraint(model2, tfin - t[1,3] >= Duree[1,3] )
@constraint(model2, tfin - t[2,3] >= Duree[2,3] )


# define x variables as CONTINUOUS (recall that it is not possible to define binary variables in Clp)
@variable(model2, 0 <= x_2_1__1_1 <= 1)
@variable(model2, 0 <= x_2_3__1_2 <= 1)
@variable(model2, 0 <= x_2_2__1_3 <= 1)
varsshouldbebinary=[x_2_1__1_1,x_2_3__1_2,x_2_2__1_3]


# define bigM constraints linking x and t variables
@constraint(model2, t[2,1] - t[1,1] >=  Duree[1,1] - bigM*x_2_1__1_1)
@constraint(model2, t[1,1] - t[2,1] >=  Duree[2,1] - bigM*(1-x_2_1__1_1))
@constraint(model2, t[2,3] - t[1,2] >=  Duree[1,2] - bigM*x_2_3__1_2)
@constraint(model2, t[1,2] - t[2,3] >=  Duree[2,3] - bigM*(1-x_2_3__1_2))
@constraint(model2, t[2,2] - t[1,3] >=  Duree[1,3] - bigM*x_2_2__1_3)
@constraint(model2, t[1,3] - t[2,2] >=  Duree[2,2] - bigM*(1-x_2_2__1_3))


println(model2)

Min tfin
Subject to
 -t[1,1] + t[1,2] >= 6.0
 -t[1,2] + t[1,3] >= 7.0
 -t[2,1] + t[2,2] >= 3.0
 -t[2,2] + t[2,3] >= 5.0
 -t[1,3] + tfin >= 4.0
 -t[2,3] + tfin >= 1.0
 -t[1,1] + t[2,1] + 26 x_2_1__1_1 >= 6.0
 t[1,1] - t[2,1] - 26 x_2_1__1_1 >= -23.0
 -t[1,2] + t[2,3] + 26 x_2_3__1_2 >= 7.0
 t[1,2] - t[2,3] - 26 x_2_3__1_2 >= -25.0
 t[2,2] - t[1,3] + 26 x_2_2__1_3 >= 4.0
 -t[2,2] + t[1,3] - 26 x_2_2__1_3 >= -21.0
 t[1,1] >= 0.0
 t[2,1] >= 0.0
 t[1,2] >= 0.0
 t[2,2] >= 0.0
 t[1,3] >= 0.0
 t[2,3] >= 0.0
 x_2_1__1_1 >= 0.0
 x_2_3__1_2 >= 0.0
 x_2_2__1_3 >= 0.0
 x_2_1__1_1 <= 1.0
 x_2_3__1_2 <= 1.0
 x_2_2__1_3 <= 1.0



In [11]:
listvars=[]
listvals=[]

BestTfin=bigM
Bestsol=[]

current_node_number=0
stop = false

while(!stop)
    
    println("\nNode number ", current_node_number, ": \n-----\n", model2)

    print("Solve : start ... ")
    status = optimize!(model2)
    println("... end")

    #println("\nSolution relax lin",); [print("\t", name(v),"=",value(v)) for v in all_variables(model2)]; 
    println(" "); println("\nSolution precedemment memorisee ", Bestsol, " avec date de fin ", BestTfin, "\n")

    TA, TO, TR, Bestsol, BestTfin = TestsSondabilite_relaxlin(model2, varsshouldbebinary, BestTfin, Bestsol)

    is_node_sondable = TA || TO || TR
    
    if(!is_node_sondable)
        listvars, listvals = SeparerNoeud_relaxlin(varsshouldbebinary, listvars, listvals)
    else
        listvars, listvals, stop = ExplorerAutreNoeud_relaxlin(listvars, listvals)
    end

    current_node_number = current_node_number + 1
end

println("\n******\n\nOptimal value = ", BestTfin, "\n\nOptimal t=", Bestsol)


Node number 0: 
-----
Min tfin
Subject to
 -t[1,1] + t[1,2] >= 6.0
 -t[1,2] + t[1,3] >= 7.0
 -t[2,1] + t[2,2] >= 3.0
 -t[2,2] + t[2,3] >= 5.0
 -t[1,3] + tfin >= 4.0
 -t[2,3] + tfin >= 1.0
 -t[1,1] + t[2,1] + 26 x_2_1__1_1 >= 6.0
 t[1,1] - t[2,1] - 26 x_2_1__1_1 >= -23.0
 -t[1,2] + t[2,3] + 26 x_2_3__1_2 >= 7.0
 t[1,2] - t[2,3] - 26 x_2_3__1_2 >= -25.0
 t[2,2] - t[1,3] + 26 x_2_2__1_3 >= 4.0
 -t[2,2] + t[1,3] - 26 x_2_2__1_3 >= -21.0
 t[1,1] >= 0.0
 t[2,1] >= 0.0
 t[1,2] >= 0.0
 t[2,2] >= 0.0
 t[1,3] >= 0.0
 t[2,3] >= 0.0
 x_2_1__1_1 >= 0.0
 x_2_3__1_2 >= 0.0
 x_2_2__1_3 >= 0.0
 x_2_1__1_1 <= 1.0
 x_2_3__1_2 <= 1.0
 x_2_2__1_3 <= 1.0

Solve : start ... ... end
 

Solution precedemment memorisee Any[] avec date de fin 26

non sondable

Node number 1: 
-----
Min tfin
Subject to
 -t[1,1] + t[1,2] >= 6.0
 -t[1,2] + t[1,3] >= 7.0
 -t[2,1] + t[2,2] >= 3.0
 -t[2,2] + t[2,3] >= 5.0
 -t[1,3] + tfin >= 4.0
 -t[2,3] + tfin >= 1.0
 -t[1,1] + t[2,1] + 26 x_2_1__1_1 >= 6.0
 t[1,1] - t[2,1] - 26 x_2_

In [12]:
function BellmanFordLongPSE(graphe, source)
    
    # Nombre de sommets du graphe
    nbSommets = size(graphe)[1]

    # Distance parcourue pour chaque
    distance =  fill(-Inf,(nbSommets + 1,nbSommets))
    distance[1, source]=0

    k=1
    bool=true
    
    # L'algorithme de Bellman-Ford s'arrête dès que 2 lignes du tableau de
    # suite sont égales ou que k vaut nbSommets + 1
    while (bool && k<nbSommets+1)
        # On incrémente k
        k += 1
        # La ligne k est initialisée à partie de la ligne k-1
        distance[k, :] = distance[k-1, :]
        for i in 2:nbSommets
            # On récupère les prédécesseurs de i
            prede_i= Predecesseur(graphe,i)
            l=[]
            for j in prede_i
                # on ajoute la somme du poids du predecesseur et de sa distance
                l=append!([distance[k-1,j]+graphe[j,i]],l)
            end
            # On récupère le maximum de la liste précédente
            distance[k,i], indice = maximumTableau(l)
        end
        # Si cette ligne du tableau est la même que la précédente on s'arrête
        if (distance[k-1, :]==distance[k, :])
                bool=false
        end
    end
    # On retourne la dernière ligne sauf le premier et le dernier élément, ainsi que la valeur maximale
    return distance[k,2:(nbSommets-1)], distance[k, nbSommets]
end

BellmanFordLongPSE (generic function with 1 method)

In [13]:
#Fonction permettant de savoir si un graphe est sondable
function grapheSondable(distance, pairesDisjonction)
    est_sondable = true
    lignes, colonnes = size(distance)
    #On vérifie que les conditions sur la partie disjonctive sont vérifiées
    #Soit que ∀(ij, ik) ∈ U, tik − tij ≥ pij
    for i in 1:lignes
        for j in 1:(colonnes - 1)
            est_sondable = est_sondable && (distance[i, j+1] - distance[i,j] >= Duree[i,j])
        end
    end
    for i in 1:length(pairesDisjonction)
        #On vérifie que les conditions sur partie disjonctive associée aux conflits d’utilisation 
        #d’une ressource non partageable sont vérifiées
        #Soit que ∀(ij, kl) ∈ D, on a soit tkl − tij ≥ pij , soit tij − tkl ≥ pkl
        x = distance[pairesDisjonction[i][2][1], pairesDisjonction[i][2][2]] - distance[pairesDisjonction[i][1][1], pairesDisjonction[i][1][2]]
        if (x > 0)
            est_sondable = est_sondable && (x >= Duree[pairesDisjonction[i][1][1], pairesDisjonction[i][1][2]])
        else
            est_sondable = est_sondable && (x >= Duree[pairesDisjonction[i][2][1], pairesDisjonction[i][2][2]])
        end
    end
    return est_sondable
end

#Fonction permettant de convernir un tableau sous forme de ligne
#en un tableau sous forme lignes/colonnes
function conversion(tableau, model)
    conversion = copy(model)
    lignes, colonnes = size(model) 
    for i in 1:lignes
        conversion[i,:]=tableau[(i-1)*colonnes+1:i*colonnes]
    end
    return conversion
end

conversion (generic function with 1 method)

In [14]:
#Fonction réalisant la PSE basé sur un graphe disjonctif
function PSEGrapheDisjonctif(graphe, disjonction, pairesDisjonction, Duree)
    BestTfin=sum(Duree)
    Bestsol=[]
    
    lignes, colonnes = size(Duree)

    current_node_number=0
    #La pile des graphes à analyser
    memoire = []
    k = 1
    #pile permettant d'indiquer quel sous-arbre est analysé
    fils = []

    push!(fils, 0)
    #On ajoute le graphe initial pour l'analyser
    push!(memoire, graphe)
    
    #Tant que toutes les possibilités n'ont pas été explorées, on continue
    while (length(memoire) >= 1)
        println("\nNode number ", current_node_number, ": \n-----")
        println("Solution precedemment memorisee ", Bestsol, " avec date de fin ", BestTfin)

        #On récupère le graphe à analyser
        grapheAnalyser=pop!(memoire)
        #On calcule les distances et la distance maximale
        distances, distanceMax = BellmanFordLongPSE(grapheAnalyser, 1)

        #On convertit le tableau pour qu'il est la même forme que Duree
        tableauConverti = conversion(distances, Duree)

        #On vérifie qu'un résultat est possible
        if (distanceMax != -Inf)
            #On regarde si le graphe est sondable et que k est inférieur ou égal au nombre de paires
            #de disjonction
            if (!grapheSondable(tableauConverti, pairesDisjonction) && k <= length(pairesDisjonction))
                
                println("Non sondable")  
                
                #On ajoute les fils du graphe qui vont être par la suite analysé
                push!(fils, 2)    
                push!(fils, 1)

                #On calcule les commets de la paire de disjonction
                sommet2 = (pairesDisjonction[k][1][1]-1)*colonnes + 1 + pairesDisjonction[k][1][2]
                sommet1 = (pairesDisjonction[k][2][1]-1)*colonnes + 1 + pairesDisjonction[k][2][2]

                #On met à jour le graphe dans le cas où la variable binaire est supérieure ou égale à 1
                #Fils de gauche
                matriceChoixArc1 = copy(grapheAnalyser)
                matriceChoixArc1[sommet1, sommet2] = disjonction[k, 2]   

                #On met à jour le graphe dans le cas où la variable binaire est inférieure ou égale à 1
                #Fils de droite
                matriceChoixArc2 = copy(grapheAnalyser)
                matriceChoixArc2[sommet2, sommet1] = disjonction[k, 1]  

                #On ajoute les deux graphes en ajoute d'abord celui avec le fils de gauche puis celui de droite
                push!(memoire, matriceChoixArc2)
                push!(memoire, matriceChoixArc1)
                
                #On incrémente k
                k = k+1
                
            else
                #Si il est sondable on regarde la distance obtenue
                if (distanceMax >= BestTfin)
                    #Test d'optimalité
                    println("TO")
                else
                    #Test de resolution
                    println("TR")
                    if (distanceMax < BestTfin)
                        #Si la solution est meilleure, on met à jour BestTfin et Bestsol
                        BestTfin = distanceMax
                        Bestsol = tableauConverti
                    end
                end
                
                #Si le premier fils rencontré vaut 2, on dépile jusqu'à temps
                #de trouver un fils qui vaut 1
                #Ainsi, on retourne à la profondeur qu'il faut analyser
                valeur = pop!(fils)
                if (valeur == 2)    
                    while (valeur == 2)
                        valeur = pop!(fils)
                        k = k-1
                    end
                end
            end
        else
            #Si le problème est infaisable on retourne TA
            println("TA")
        end
        #On incrément le noeud courant
        current_node_number = current_node_number + 1
    end
    #Fini
    println("\nFINISHED")
    #On retourne BestTfin et Bestsol
    return BestTfin, Bestsol
end

PSEGrapheDisjonctif (generic function with 1 method)

In [15]:
Duree=[6 7 4; 3 5 1]

graphe = 
[#debut a1    a2     a3     b1    b2      b3     fin
  0     0     -Inf   -Inf   0     -Inf    -Inf   -Inf ;
 -Inf   0     6      -Inf   -Inf   -Inf   -Inf   -Inf ;
 -Inf   -Inf  0      7      -Inf   -Inf   -Inf   -Inf ;
 -Inf   -Inf  -Inf   0      -Inf   -Inf   -Inf   4 ;
 -Inf   -Inf  -Inf   -Inf   0     3       -Inf   -Inf ;
 -Inf   -Inf  -Inf   -Inf   -Inf  0       5      -Inf ;
 -Inf   -Inf  -Inf   -Inf   -Inf  -Inf    0      1 ;
 -Inf   -Inf  -Inf   -Inf   -Inf  -Inf    -Inf   0 ;
]

valeursDisjonction = [ 6 3 ; 7 1 ; 4 5 ]
pairesDisjonction = [((1,1),(2,1)), ((1,2),(2,3)), ((1,3),(2,2))]

BestTfin, Bestsol = PSEGrapheDisjonctif(graphe, valeursDisjonction, pairesDisjonction, Duree)

println("\n******\n\nOptimal value = ", BestTfin, "\n\nOptimal t=", Bestsol)


Node number 0: 
-----
Solution precedemment memorisee Any[] avec date de fin 26
Non sondable

Node number 1: 
-----
Solution precedemment memorisee Any[] avec date de fin 26
Non sondable

Node number 2: 
-----
Solution precedemment memorisee Any[] avec date de fin 26
Non sondable

Node number 3: 
-----
Solution precedemment memorisee Any[] avec date de fin 26
TR

Node number 4: 
-----
Solution precedemment memorisee [3 9 16; 0 3 8] avec date de fin 20.0
TO

Node number 5: 
-----
Solution precedemment memorisee [3 9 16; 0 3 8] avec date de fin 20.0
Non sondable

Node number 6: 
-----
Solution precedemment memorisee [3 9 16; 0 3 8] avec date de fin 20.0
TO

Node number 7: 
-----
Solution precedemment memorisee [3 9 16; 0 3 8] avec date de fin 20.0
TO

Node number 8: 
-----
Solution precedemment memorisee [3 9 16; 0 3 8] avec date de fin 20.0
Non sondable

Node number 9: 
-----
Solution precedemment memorisee [3 9 16; 0 3 8] avec date de fin 20.0
Non sondable

Node number 10: 
-----
Solu

In [16]:
Duree=[6 7 0; 3 5 1]

graphe = 
[#debut a1    a2     a3     b1    b2      b3     fin
  0     0     -Inf   -Inf   0     -Inf    -Inf   -Inf ;
 -Inf   0     6      -Inf   -Inf   -Inf   -Inf   -Inf ;
 -Inf   -Inf  0      7      -Inf   -Inf   -Inf   -Inf ;
 -Inf   -Inf  -Inf   0      -Inf   -Inf   -Inf   0 ;
 -Inf   -Inf  -Inf   -Inf   0     3       -Inf   -Inf ;
 -Inf   -Inf  -Inf   -Inf   -Inf  0       5      -Inf ;
 -Inf   -Inf  -Inf   -Inf   -Inf  -Inf    0      1 ;
 -Inf   -Inf  -Inf   -Inf   -Inf  -Inf    -Inf   0 ;
]

valeursDisjonction = [ 6 3 ; 7 1 ]
pairesDisjonction = [((1,1),(2,1)), ((1,2),(2,3))]

BestTfin, Bestsol = PSEGrapheDisjonctif(graphe, valeursDisjonction, pairesDisjonction, Duree)

println("\n******\n\nOptimal value = ", BestTfin, "\n\nOptimal t=", Bestsol)


Node number 0: 
-----
Solution precedemment memorisee Any[] avec date de fin 22
Non sondable

Node number 1: 
-----
Solution precedemment memorisee Any[] avec date de fin 22
Non sondable

Node number 2: 
-----
Solution precedemment memorisee Any[] avec date de fin 22
TR

Node number 3: 
-----
Solution precedemment memorisee [3 9 16; 0 3 8] avec date de fin 16.0
TO

Node number 4: 
-----
Solution precedemment memorisee [3 9 16; 0 3 8] avec date de fin 16.0
TR

FINISHED

******

Optimal value = 15.0

Optimal t=[0 6 13; 6 9 14]
